In [3]:
import YanAPI as YanAPI
from mutagen.mp3 import MP3
import time

ip_addr = "192.168.1.12" # please change to your yanshee robot IP
YanAPI.yan_api_init(ip_addr)


In [5]:
import yt_dlp
from youtube_search import YoutubeSearch

def download_music(search_str):
    YTSearchResult = YoutubeSearch(search_str, max_results=1).to_dict()

    URLS = "https://www.youtube.com{a}".format(a=YTSearchResult[0]['url_suffix'])

    ydl_opts = {
        'format': 'mp3/bestaudio/best',
        'ffmpeg_location': '.venv/Lib/site-packages/ffmpeg-full_build/bin',
        'outtmpl': './Music/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(URLS)
    
    return ydl_opts['outtmpl']

SyntaxError: invalid syntax (<ipython-input-5-c6a095fc09e1>, line 7)

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

def find_music(search_str):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id="b89acd4356eb4d0ca884d29d568daa20",
        client_secret="ec186bc7b2fc4aeeb4afd41afda26cad"
        )
    )

    results = sp.search(search_str)

    search_result=[]
    for result in results['tracks']['items']:
            
        AssistArtist = ""
        for i in range(1,len(result['artists'])):
            temp = result['artists'][i]['name']
            if i == 1:
                AssistArtist += temp
            else:
                AssistArtist = ", " + temp
            
        result_details = dict(
            SongName = result['name'],
            MainArtist = result['artists'][0]['name'],
            FeaturedArtist = AssistArtist,
            URI = result['uri']
        )
        search_result.append(result_details)

    result = pd.DataFrame(search_result)

    result.to_csv('./Result.csv')
    print(result)

    search_str = str(result['SongName'][0] + ' ' + result['MainArtist'][0])

    return search_str

In [ ]:
def listen():
    while True:
        YanAPI.start_voice_tts("What do want to listen to",False)
        time.sleep(2) #sleep 2 seconds to ensure the voice will not be interapt by the code below
        listen_res = YanAPI.sync_do_voice_asr_value() # listen again to get the name.
        if len(listen_res["answer"]) > 0:# user said something
            YanAPI.start_voice_tts("Please wait while I find the song",False)
            time.sleep(3)
            result = find_music(listen_res['answer'])
            YanAPI.start_voice_tts("Is this what you are looking for: "+result+"",False)
            if YanAPI.sync_do_voice_asr_value()['answer'] == 'no':
                continue
            else:
                YanAPI.start_voice_tts("Please wait while I download the song",False)
                break
        else:
            YanAPI.start_voice_tts("I cannot hear you, try again",False)
    return listen_res['answer']

In [ ]:
def play_song():
    title = download_music(listen())
    YanAPI.start_play_music(f'./Music/{title}')
    time.sleep(5)
    # time.sleep(MP3(f'./Music/{title}').info.length)
    YanAPI.stop_play_music()

In [ ]:
play_song()